# Chapter 4: Models

See also [Writing your first Django app, part 2](https://docs.djangoproject.com/en/5.0/intro/tutorial02/).

## Database setup

Database settings in `settings.py`:

In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

Run initial migrations:

Create superuser:

## Creating a simple model

Create model `Post` in `models.py`:

In [ ]:
from django.db import models

# Create your models here.

class Post(models.Model):
    title = models.CharField(max_length=100)
    text = models.TextField(max_length=1000)

Register model in `admin.py`:

In [ ]:
from django.contrib import admin

from .models import Post

# Register your models here.

admin.site.register(Post)

Create and execute migrations.

Start development server

Explore admin interface at http://localhost:8000/admin

## Showing data in views

Create new function in `views.py`:

In [ ]:
from django.shortcuts import render

from .models import Post # NEW

# Create your views here.

def ex01_post_list(request):
    return render(
        request,
        'chp04/ex01_post_list.html'
        )

Add a new path in `urls.py`:

In [ ]:
from django.urls import path

from chp04_models import views # NEW

urlpatterns = [
    path( # NEW
        'posts/',
        views.ex01_post_list,
        name='ex01_post_list'
        ),
]

Add context to `ex01_list_posts` view in `views.py`:

In [ ]:
def ex01_list_posts(request):
    context = { # NEW
        'posts_list': []
    }
    return render(
        request,
        'chp04/ex01_post_list.html',
        context # NEW
        )

Import `Post` model in `views.py`:

In [ ]:
from django.shortcuts import render

from .models import Post # NEW

# ...

Load posts from database in `views.py`:

In [ ]:
def ex01_list_posts(request):
    posts = Post.objects.all() # NEW
    context = {
        'posts_list': posts # NEW
    }
    return render(
        request,
        'chp04/ex01_post_list.html',
        context
        )

Create new view in `views.py`:

In [ ]:
def ex02_post_detail(request, id):
    post = Post.objects.get(pk=id)
    context = {
        'post': post
    }
    return render(
        request,
        'chp04/ex02_post_detail.html',
        context
    )

Add a new path in `urls.py`:

In [ ]:
# ...

urlpatterns = [
    # ...
    path( # NEW
        'posts/<int:id>', # path parameter
        views.ex02_post_detail,
        name='ex02_post_detail'
        ),
]

## Creating relationships

Different types of relations:

* one-to-one
* one-to-many
* many-to-many

### one-to-one

Example: User and user profile

**One** user has only **one** profile,
**one** profile belongs to **only one** user.

### one-to-many

Example: Author and comment

**One** author writes **many** comments,
but **one** comment belongs to **only one** author.

### many-to-many

Example: Posts and keywords

**One** post has **many** keywords,
**one** keyword belongs to **many** posts.

Create model `Category` in `models.py` **above** the `Post` model:

In [ ]:
from django.db import models

# Create your models here.

class Category(models.Model):
    title = models.CharField(max_length=100)

# ...

Register model in `admin.py`:

In [ ]:
from django.contrib import admin

from .models import Category, Post # NEW

# Register your models here.

admin.site.register(Category) # NEW
admin.site.register(Post)

Add relation `category` to model `Post`.

In [ ]:
# ...

class Post(models.Model):
    title = models.CharField(max_length=100)
    text = models.TextField(max_length=1000)
    category = models.ForeignKey(
        Category,
        on_delete=models.SET_NULL,
        null=True, # allow empty category
        )

# ...

Create and execute migrations: